#Assigment 3: Social Network Analysis for U.S presidential elections
<br>42913 Social and Information Network Analysis - Autumn 2020

<br>Carlos Mario Carvajal Moreno. Student ID: 13144148
<br>Andres Felipe Lagos. Student ID: 13092248
<br>Ernest Ilustre.  Student ID: 12763239




# The Twitter API and Authentication


To access the twitter API, it is required to have twitter developer account. With this account the following authentication credentials can be obtained and will be used for streaming tweets in python from the tweeter API:

* API Key
* API Secret
* Access Token 
* Access Token Secret

However, as the credentials belong to a personal account, the following code will not display them for privacy and security matters. To run this code, please use your own credential or contact one of the owners of this notebook to have a live demonstration.

## Tweeter Streaming APIs

This API will be used for straming and reading data from twitter. Any public twitter ac in particular we will use the public streams which twitter documentation defines as stream of the public data flowing through tweeter.

The following code stream Tweets and retrieve the data in a JSON format that contain great number fields. However, only few will be studied by producing filterings.

In [1]:
pip install twython

In [0]:
import tweepy, json
import pandas as pd
from twython import Twython
import csv
import os

#Please use your twitter developer credentials to run this code
access_token = ""
access_token_secret = ""
consumer_key = ""
consumer_secret = ""

#pass the api key and secret to the handler
#pass the credentials using the set_access_token method
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)


##Defining the twitter stream listener class.

in the following code a twitter listener has been generated to create a file that called tweets.txt. This listener collects streaming tweets and writtes it to the file tweets.text. In it we define the number of tweets to be streamed and after it finishes streaming the tweets the listener closes the file and stops listening.


In [0]:
class MyStreamListener(tweepy.StreamListener):
    def __init__(self, api=None):
        super(MyStreamListener, self).__init__()
        self.num_tweets = 0
        self.file = open("tweets.txt", "w")

    def on_status(self, status):
        tweet = status._json
        if not tweet['retweeted'] and 'RT @' not in tweet['text']:
          if "extended_tweet" in tweet:
            if tweet['user']['followers_count'] > 1:
              self.file.write( json.dumps(tweet) + '\n' )
              self.num_tweets += 1
              if self.num_tweets < 1000000:
                return True
              else:
                return False
              self.file.close()

    def on_error(self, status):
        print(status)


api = tweepy.API(auth)

After writting the listener class, the streaming object needs authentication, and will be passed a query to stream tweets.

## Streaming Tweets and saving them in a txt file

In [0]:
l = MyStreamListener()

key_words = ['trump', 'biden', 'maga', 'makeamericagreatagain','nevertrump','trump2020','biden2020',
             'Joe Biden', '2020 elections', 'trump president','neverbiden', 'biden president']

stream = tweepy.Stream(auth, l, tweet_mode='extended', query = {'q': '-filter:RT @',
                                                               'language': 'en'} )

stream.filter(track = key_words) 

The following code will set .

In [0]:
# String of path to file: tweets_data_path
tweets_data_path = 'tweets.txt'

# Initialize empty list to store tweets: tweets_data
tweets_data = []

# Open connection to file
tweets_file = open(tweets_data_path, "r")

In [0]:
# Read in tweets and store in list: tweets_data
for line in tweets_file:
    tweet = json.loads(line)
    tweets_data.append(tweet)

# Close connection to file
tweets_file.close()

# Creating an empty dictionary to append the tweets
dict_ = {'Tweet_id': [], 'User_id':[], 'user': [], 'date': [], 'text': [], 'favourites_count': [],
         "followers_count": [], "Retweet_count":[], 'Place':[]
         }

# For loop to append data into the dictionary        
for status in tweets_data:
    dict_['Tweet_id'].append(status['id'])
    dict_['user'].append(status['user']['screen_name'])
    dict_['User_id'].append(status['user']['id_str'])
    dict_['date'].append(status['created_at'])
    if 'extended_tweet' in status.keys():
      dict_['text'].append(status['extended_tweet']['full_text'])
    else:
      dict_['text'].append(status['text'])
    dict_['favourites_count'].append(status['favorite_count'])
    dict_['followers_count'].append(status['user']['followers_count'])
    dict_['Retweet_count'].append(status['retweet_count'])
    dict_['Place'].append(status['user']['location'])

# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)
df.sort_values(by='favourites_count', inplace=True, ascending=False)
df.head()

,Tweet_id,User_id,user,date,text,favourites_count,followers_count,Retweet_count,Place
0,1269552454628249600,37755581,JohnHopkins_,Sun Jun 07 08:51:06 +0000 2020,"@DelGeezer 👁️ We see you there, we know what y...",0,1435,0,Liverpool UK
26255,1269595199652728834,47331230,Zebra1944,Sun Jun 07 11:40:58 +0000 2020,PBOTUS=PUNK BULLY OF THE UNITED STATES=DONALD ...,0,14,0,Pennsylvania USA
26248,1269595193503858688,896904160305655808,Sherathorington,Sun Jun 07 11:40:56 +0000 2020,@dooleyynoted they don't tell you these things...,0,1952,0,None
26249,1269595193877114882,516645637,ianbodgerbrown,Sun Jun 07 11:40:56 +0000 2020,Can Congress Impeach Trump Again For His Worst...,0,137,0,"Manchester, UK"
26250,1269595195957534722,1707512646,fmiller26,Sun Jun 07 11:40:57 +0000 2020,@MZHemingway Every Democrat should be voted ou...,0,2324,0,"Indiana, USA"


In [0]:
len(df)

39378

In [0]:
df['Place'].value_counts()

United States                 993
USA                           339
Florida, USA                  280
Texas, USA                    218
London, England               194
                             ... 
Wisconsin and Nevada            1
ישראל                           1
Buford, GA                      1
Great Plains                    1
Somewhere in space & time.      1
Name: Place, Length: 9017, dtype: int64

In [0]:
from google.colab import files
df.to_csv('tweeter_dataset.csv')

files.download('tweeter_dataset.csv')

## Extracting the most popular tweets from a region using Twython

In [0]:
# Instantiate an object
python_tweets = Twython(consumer_key, consumer_secret)

key_words = ['trump', 'biden', 'maga', 'makeamericagreatagain','nevertrump','trump2020','biden2020',
             '2020 elections','neverbiden']

result_type = ['popular','recent','mixed']

list_id = []

 # Creating empty dictionary
dict_ = {'Tweet_id': [], 'User_id':[], 'user': [], 'date': [], 'text': [], 'favorite_count': [], "followers_count": [],
         'Retweeted': [], "Retweet_count":[], 'Place':[]
         }


for word in key_words:
  for result in result_type:
    for id_ in list_id:
      query = {'q': word,
               'result_type': result_type,
               'count' : 100,
               'lang': 'en',
               'since_id': list_id}
        
        for status in python_tweets.search(**query, tweet_mode='extended')['statuses']:
          dict_['Tweet_id'].append(status['id'])
          dict_['user'].append(status['user']['screen_name'])
          dict_['User_id'].append(status['user']['id_str'])
          dict_['date'].append(status['created_at'])
          dict_['text'].append(status['full_text'])
          dict_['favorite_count'].append(status['favorite_count'])
          dict_['followers_count'].append(status['user']['followers_count'])
          dict_['Retweeted'].append(status['retweeted'])
          dict_['Retweet_count'].append(status['retweet_count'])
          dict_['Place'].append(status['user']['location'])

# Structure data in a pandas DataFrame for easier manipulation
df1 = pd.DataFrame(dict_)
df1.sort_values(by='favorite_count', inplace=True, ascending=False)
df1.head()

In [0]:
# second part
df1.to_csv('tweets.csv')

files.download('tweets.csv')